In [124]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [125]:
url = "https://stackoverflow.com/jobs/companies"

connection_attempts = 3

while connection_attempts > 0:
    response = requests.get(url)

    if response.status_code == 200:
        print("Successfully connected")
        soup = BeautifulSoup(response.text, "html")
        print(soup)
        break

    elif response.status_code == 429:
         print("Response [429], Reattempting to connect")
         connection_attempts -= 1

    elif response.status_code == 403:
            print("Access denied") 
            break
    else:
         print("Invalid link")
         break




Successfully connected
<!DOCTYPE html>
<html class="html__responsive" lang="en">
<head>
<title>Best Companies Hiring Developers  - Stack Overflow</title>
<link href="https://cdn.sstatic.net/Sites/stackoverflow/Img/favicon.ico?v=ec617d715196" rel="shortcut icon"/>
<link href="https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon.png?v=c78bd457575a" rel="apple-touch-icon"/>
<link href="https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon.png?v=c78bd457575a" rel="image_src"/>
<link href="/opensearch.xml" rel="search" title="Stack Overflow" type="application/opensearchdescription+xml"/>
<meta content="width=device-width, height=device-height, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<meta content="noarchive" name="bingbot"/>
<script id="webpack-public-path" type="text/uri-list">https://cdn.sstatic.net/</script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.7.1/jquery.min.js"></script><script src="https://code.jquery.com/jquery-migrate-3.4

In [148]:
url = "https://stackoverflow.com/jobs/companies/logius-onderdeel-ministerie-bzk"
re = requests.get(url)
sup = BeautifulSoup(re.text, "html")
sup

<!DOCTYPE html>
<html class="html__responsive" lang="en">
<head>
<title>Logius, onderdeel ministerie BZK   - Stack Overflow</title>
<link href="https://cdn.sstatic.net/Sites/stackoverflow/Img/favicon.ico?v=ec617d715196" rel="shortcut icon"/>
<link href="https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon.png?v=c78bd457575a" rel="apple-touch-icon"/>
<link href="https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon.png?v=c78bd457575a" rel="image_src"/>
<link href="/opensearch.xml" rel="search" title="Stack Overflow" type="application/opensearchdescription+xml"/>
<meta content="width=device-width, height=device-height, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<meta content="noarchive" name="bingbot"/>
<script id="webpack-public-path" type="text/uri-list">https://cdn.sstatic.net/</script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.7.1/jquery.min.js"></script><script src="https://code.jquery.com/jquery-migrate-3.4.1.min.js"></script>
<

In [159]:
x = sup.find_all("span", class_="d-block")

for item in range(12, len(x)+1):

    print(x[item].text.strip())


Computer Software, Government, IT Consulting
501-1k employees
Public
44


IndexError: list index out of range

In [ ]:
for item in range(12, len(x)+1):
    try:
        print(x[item].text.strip())
    except IndexError:
        pass


In [168]:
# TEST - Stopped here
def add_data_to_df(soupy):
    company_name = soupy.find("h1").text.strip() # Company Name

    
    about_company = soupy.find_all("span", class_ = "d-block")




    x = sup.find_all("span", class_="d-block")

    

    indices = [12,13,14,15,16]
    values = []

    for index in indices:
        try:
            values.append(about_company[index].text.strip())
        except IndexError:
            values.append("Null")


    industry = values[0]
    size = values[1]
    year_founded = values[2]
    status = values[3]
    followers = values[4]








    tech_skills = soupy.find_all("a", class_ = "flex--item s-tag no-tag-menu") # Searched for the company's tech stack
    tech_stack = [] # Created a list containing all relevant tech skills 

    for item in range(len(tech_skills)):
        tech_stack.append(tech_skills[item].text.strip())


    benefits_list = soupy.find_all("div", class_="flex--item pl8 pt2 fw-normal fs-body2 fc-black-700") # Searched for company benefits
    benefits = [] # Created a list containing all listed company benefits

    for item in range(len(benefits_list)):
        benefits.append(benefits_list[item].text.strip())
    
    
    # Appended all webscrapped elements into a list "new_row"

    new_row = [] 

    new_row.append(company_name)
    new_row.append(industry)
    new_row.append(size)
    new_row.append(year_founded)
    new_row.append(status)
    new_row.append(followers)
    new_row.append(tech_stack)
    new_row.append(benefits)

    df.loc[len(df.index)] = new_row # Added the list as a new row in the dataframe

In [177]:
# Scraping Multiple Pages

connection_attempts = 3
max_pages = 7

for page in range(1, max_pages + 1):
    url = f"https://stackoverflow.com/jobs/companies?pg={page}"
     
    while connection_attempts > 0:
        response = requests.get(url)

        if response.status_code == 200:
            print(f"Successfully connected to page:{page}")
            soup = BeautifulSoup(response.text, "html")
            scrape_page(soup)
            break

        elif response.status_code == 429:
            print("Response [429]: Reattempting to connect")
            connection_attempts -= 1

        elif response.status_code == 403:
                print("Response [403]: Access denied") 
                break
        else:
            print("Invalid link")
            break
    if connection_attempts == 0:
        print("Max number of attempts excceded")

Successfully connected to page:1
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:2
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:3
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:4
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful

In [178]:
display(df)

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
0,Intuit,"Computer Software, Financial Technology",10k+ employees,1983,Public,642,"[java, kotlin, scala, play, spark, react, aws,...","[Well-being for Life Reimbursement Program, Em..."
1,Discover Financial Svcs,"Banking, Financial Services",10k+ employees,1986,Public,150,"[javascript, tableau-api, powerbi, qlikview, t...","[Annual Leave - 5 to 7 weeks, Tuition Reimburs..."
2,Discover Financial Services,"Banking, Financial Services, Financial Technology",10k+ employees,1985,Public,180,"[tableau-api, powerbi, qlikview, teradata, had...","[Paid Time Off - 4 to 6 weeks per year, Health..."
3,Global Industrial,"B2B Sales, eCommerce, Information Technology",1k-5k employees,1949,Public,105,"[java, jakarta-ee, spring, reactjs, next.js, j...",[Employee Stock Purchase Plan – 15% discount o...
4,ALDI SÜD IT,"E-Commerce, Information Technology, Retail",1k-5k employees,1913,Private,45,"[angular, asp.net-core, azure, bash, c#, conti...","[30 days of annual leave, Holiday and Christma..."
...,...,...,...,...,...,...,...,...
59,UBS,"Financial Services, Financial Technology",10k+ employees,Private,102,Null,"[javascript, java, kotlin, kubernetes, cicd, p...",[Working with cutting edge tech and agile prac...
60,ZEISS Group,"Machine Learning, Medical Devices, Virtual Rea...",10k+ employees,1846,Private,218,"[angular, swift, c#, azure, javascript, python...","[Flexible working time models, Open, dialogue-..."
61,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,Public,192,Null,"[java, python, api, management, c#, ai, cloud-...","[Medical, Dental and Vision coverage, Paid Tim..."
62,KfW,5k-10k employees,1948,Public,52,Null,"[java, pl1, javascript, python, sap, c#, c++, ...","[Marktgerechte Vergütung, Betriebliche Altersv..."


## Scraping each company profile page

In [176]:
df = pd.DataFrame(
    columns = [["company", "industry", "size", "year_founded", "status", "follower_count", "tech_stack", "benefits"]]
)


In [144]:
# Creating the def function

def add_data_to_df(soupy):
    company_name = soupy.find("h1").text.strip() # Company Name

    
    about_company = soupy.find_all("span", class_ = "d-block")

    industry = about_company[12].text.strip() # Narrowed search to only industry, size, year founded, status, and followers by calling specific indexes
    size = about_company[13].text.strip() 
    year_founded = about_company[14].text.strip()
    status = about_company[15].text.strip()
    followers = about_company[16].text.strip()


    tech_skills = soupy.find_all("a", class_ = "flex--item s-tag no-tag-menu") # Searched for the company's tech stack
    tech_stack = [] # Created a list containing all relevant tech skills 

    for item in range(len(tech_skills)):
        tech_stack.append(tech_skills[item].text.strip())


    benefits_list = soupy.find_all("div", class_="flex--item pl8 pt2 fw-normal fs-body2 fc-black-700") # Searched for company benefits
    benefits = [] # Created a list containing all listed company benefits

    for item in range(len(benefits_list)):
        benefits.append(benefits_list[item].text.strip())
    
    
    # Appended all webscrapped elements into a list "new_row"

    new_row = [] 

    new_row.append(company_name)
    new_row.append(industry)
    new_row.append(size)
    new_row.append(year_founded)
    new_row.append(status)
    new_row.append(followers)
    new_row.append(tech_stack)
    new_row.append(benefits)

    df.loc[len(df.index)] = new_row # Added the list as a new row in the dataframe


In [143]:
# entering each link 

def scrape_page(page_link):

    company_list = page_link.find_all("a", class_="s-link", href=lambda href: href and "/jobs/companies" in href)

    company_links = []

    for names in range(len(company_list)):
        company_links.append("https://stackoverflow.com" + company_list[names]["href"])



    for link in range(len(company_links)):  # currently testing -> final code would have range(len(company_links))
        response = requests.get(company_links[link])
        max_attempts = 3

        while max_attempts > 0:

            if response.status_code == 200:
                print("Successful connection")
                soupy = BeautifulSoup(response.text, "html")
                add_data_to_df(soupy)
                break

            elif response.status_code == 429:
                print("Response 429, reattempting...")
                max_attempts -=1

            elif response.status_code == 403:
                print("Access Denied")
                break
            
            else:
                print(response.status_code)
                break

In [147]:
df

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
0,Intuit,"Computer Software, Financial Technology",10k+ employees,1983,Public,642,"[java, kotlin, scala, play, spark, react, aws,...","[Well-being for Life Reimbursement Program, Em..."
1,Discover Financial Svcs,"Banking, Financial Services",10k+ employees,1986,Public,150,"[javascript, tableau-api, powerbi, qlikview, t...","[Annual Leave - 5 to 7 weeks, Tuition Reimburs..."
2,Discover Financial Services,"Banking, Financial Services, Financial Technology",10k+ employees,1985,Public,180,"[tableau-api, powerbi, qlikview, teradata, had...","[Paid Time Off - 4 to 6 weeks per year, Health..."
3,Global Industrial,"B2B Sales, eCommerce, Information Technology",1k-5k employees,1949,Public,105,"[java, jakarta-ee, spring, reactjs, next.js, j...",[Employee Stock Purchase Plan – 15% discount o...
4,ALDI SÜD IT,"E-Commerce, Information Technology, Retail",1k-5k employees,1913,Private,45,"[angular, asp.net-core, azure, bash, c#, conti...","[30 days of annual leave, Holiday and Christma..."
5,IT Sonix Custom Development GmbH,"Agile Software Development, Software Consultin...",51-200 employees,2011,Private,163,"[java, spring, boot, .net, core, c#, angular, ...","[Weiterbildung, Flexible Arbeitszeiten, Mobile..."
6,JPMorgan Chase & Co.,Financial Services,10k+ employees,1799,Public,1.1k,"[java, python, sql, devops, bigdata, mobile, c...","[Healthcare and Insurance Plans, Wellness Prog..."
7,adjoe,"Ad Tech, Advertising Technology, Mobile Applic...",51-200 employees,2018,Private,364,"[ios, android, mobile, kotlin, elasticsearch, ...","[Work with the newest technologies, Free in-ho..."
8,EMBL-EBI (EMBL's European Bioinformatics Insti...,"Big Data, Data Science, Life Sciences",501-1k employees,1992,Public,156,"[javascript, java, angular, reactjs, node.js, ...","[Private Health Insurance, 30 days annual leav..."
9,Sartorius,"Biotechnology, Life Sciences",10k+ employees,1870,Private,205,"[amazon-web-services, c, c#, c++, java, javasc...","[Collaboration in international project teams,..."
